# Прогнозирование стоимости жизни клиента (CLTV)

Наш путь к успеху:
1. Определить подходящий временной интервал для расчета стоимости жизни клиента (Customer Lifetime Value, CLTV).
2. Определить признаки, которые мы собираемся использовать для прогнозирования будущего, создать их.
3. Рассчитать стоимость жизни клиента (LTV) для обучения модели машинного обучения.
4. Создать и запустить модель машинного обучения.
5. Проверить, полезна ли модель.

Выбор временного интервала зависит от вашей отрасли, бизнес-модели, стратегии и других факторов. Для некоторых отраслей 1 год - очень долгий период, в то время как для других это очень короткий срок. В нашем примере мы используем 6 месяцев.

RFM Score для каждого идентификатора клиента - это отличный кандидат для набора признаков.

Нам нужно разделить наш набор данных, чтобы правильно реализовать RFM. Мы возьмем данные за 3 месяца, рассчитаем RFM Score и будем использовать их для прогнозирования следующих 6 месяцев. Таким образом, нам сначала нужно создать два набора данных и добавить в них значения RFM.

In [1]:
#import libraries
from datetime import datetime, timedelta,date
import pandas as pd
%matplotlib inline
from sklearn.metrics import classification_report,confusion_matrix
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from __future__ import division
from sklearn.cluster import KMeans

import plotly
import plotly.graph_objs as go

import xgboost as xgb
from sklearn.model_selection import KFold, cross_val_score, train_test_split

In [40]:
#read data from csv and redo the data work we done before
tx_data = pd.read_csv('../../data/OnlineRetail.csv', encoding='cp1252')
tx_data['InvoiceDate'] = pd.to_datetime(tx_data['InvoiceDate'])
tx_uk = tx_data.query("Country=='United Kingdom'").reset_index(drop=True)

In [41]:
tx_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [42]:
#create 3m and 6m dataframes
tx_3m = tx_uk[(tx_uk.InvoiceDate < "2011-06-01") & (tx_uk.InvoiceDate >= "2011-03-01")].reset_index(drop=True)
tx_6m = tx_uk[(tx_uk.InvoiceDate >= "2011-06-01") & (tx_uk.InvoiceDate < "2011-12-01")].reset_index(drop=True)
tx_3m.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,545220,21955,DOORMAT UNION JACK GUNS AND ROSES,2,2011-03-01 08:30:00,7.95,14620.0,United Kingdom
1,545220,48194,DOORMAT HEARTS,2,2011-03-01 08:30:00,7.95,14620.0,United Kingdom
2,545220,22556,PLASTERS IN TIN CIRCUS PARADE,12,2011-03-01 08:30:00,1.65,14620.0,United Kingdom
3,545220,22139,RETROSPOT TEA SET CERAMIC 11 PC,3,2011-03-01 08:30:00,4.95,14620.0,United Kingdom
4,545220,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,4,2011-03-01 08:30:00,3.75,14620.0,United Kingdom


In [43]:
#create tx_user for assigning clustering
tx_user = pd.DataFrame(tx_3m['CustomerID'].unique())
tx_user.columns = ['CustomerID']
tx_user.head()

,CustomerID
0,14620.0
1,14740.0
2,13880.0
3,16462.0
4,17068.0


In [44]:
#order cluster method
def order_cluster(cluster_field_name, target_field_name,df,ascending):
    new_cluster_field_name = 'new_' + cluster_field_name
    df_new = df.groupby(cluster_field_name)[target_field_name].mean().reset_index()
    df_new = df_new.sort_values(by=target_field_name,ascending=ascending).reset_index(drop=True)
    df_new['index'] = df_new.index
    df_final = pd.merge(df,df_new[[cluster_field_name,'index']], on=cluster_field_name)
    df_final = df_final.drop([cluster_field_name],axis=1)
    df_final = df_final.rename(columns={"index":cluster_field_name})
    return df_final

In [45]:
#calculate recency score
tx_max_purchase = tx_3m.groupby('CustomerID').InvoiceDate.max().reset_index()
tx_max_purchase.columns = ['CustomerID','MaxPurchaseDate']
tx_max_purchase['Recency'] = (tx_max_purchase['MaxPurchaseDate'].max() - tx_max_purchase['MaxPurchaseDate']).dt.days
tx_user = pd.merge(tx_user, tx_max_purchase[['CustomerID','Recency']], on='CustomerID')
tx_user.head()

,CustomerID,Recency
0,14620.0,12
1,14740.0,4
2,13880.0,25
3,16462.0,91
4,17068.0,11


In [46]:
kmeans = KMeans(n_clusters=4)
kmeans.fit(tx_user[['Recency']])
tx_user['RecencyCluster'] = kmeans.predict(tx_user[['Recency']])
tx_user = order_cluster('RecencyCluster', 'Recency',tx_user,False)
tx_user.head()

,CustomerID,Recency,RecencyCluster
0,14620.0,12,3
1,14740.0,4,3
2,17068.0,11,3
3,12971.0,4,3
4,15194.0,6,3


In [47]:
tx_user.groupby("RecencyCluster").CustomerID.count()

RecencyCluster
0    349
1    439
2    444
3    608
Name: CustomerID, dtype: int64

In [48]:
#calcuate frequency score
tx_frequency = tx_3m.groupby('CustomerID').InvoiceDate.count().reset_index()
tx_frequency.columns = ['CustomerID','Frequency']
tx_user = pd.merge(tx_user, tx_frequency, on='CustomerID')

kmeans = KMeans(n_clusters=4)
kmeans.fit(tx_user[['Frequency']])
tx_user['FrequencyCluster'] = kmeans.predict(tx_user[['Frequency']])

tx_user = order_cluster('FrequencyCluster', 'Frequency',tx_user,True)
tx_user.head()

,CustomerID,Recency,RecencyCluster,Frequency,FrequencyCluster
0,14620.0,12,3,30,0
1,15194.0,6,3,64,0
2,18044.0,5,3,57,0
3,18075.0,12,3,35,0
4,15241.0,0,3,64,0


In [49]:
tx_user.groupby("FrequencyCluster").CustomerID.count()

FrequencyCluster
0    1605
1     223
2      11
3       1
Name: CustomerID, dtype: int64

In [50]:
#calcuate revenue score
tx_3m['Revenue'] = tx_3m['UnitPrice'] * tx_3m['Quantity']
tx_revenue = tx_3m.groupby('CustomerID').Revenue.sum().reset_index()
tx_user = pd.merge(tx_user, tx_revenue, on='CustomerID')

kmeans = KMeans(n_clusters=4)
kmeans.fit(tx_user[['Revenue']])
tx_user['RevenueCluster'] = kmeans.predict(tx_user[['Revenue']])
tx_user = order_cluster('RevenueCluster', 'Revenue',tx_user,True)
tx_user.head()

,CustomerID,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster
0,14620.0,12,3,30,0,393.28,0
1,15194.0,6,3,64,0,1439.02,0
2,18044.0,5,3,57,0,808.96,0
3,18075.0,12,3,35,0,638.12,0
4,15241.0,0,3,64,0,947.55,0


In [51]:
tx_user.groupby("RevenueCluster").CustomerID.count()

RevenueCluster
0    1756
1      72
2      10
3       2
Name: CustomerID, dtype: int64

In [52]:
#overall scoring
tx_user['OverallScore'] = tx_user['RecencyCluster'] + tx_user['FrequencyCluster'] + tx_user['RevenueCluster']
tx_user['Segment'] = 'Low-Value'
tx_user.loc[tx_user['OverallScore']>2,'Segment'] = 'Mid-Value'
tx_user.loc[tx_user['OverallScore']>4,'Segment'] = 'High-Value'
tx_user.head()

,CustomerID,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster,OverallScore,Segment
0,14620.0,12,3,30,0,393.28,0,3,Mid-Value
1,15194.0,6,3,64,0,1439.02,0,3,Mid-Value
2,18044.0,5,3,57,0,808.96,0,3,Mid-Value
3,18075.0,12,3,35,0,638.12,0,3,Mid-Value
4,15241.0,0,3,64,0,947.55,0,3,Mid-Value


Поскольку наш набор функций готов, давайте рассчитаем LTV (пожизненная ценность клиента) за 6 месяцев для каждого клиента,
который мы собираемся использовать для обучения нашей модели.

В наборе данных не указаны расходы. Поэтому выручка напрямую становится нашей LTV.

In [53]:
tx_6m.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,555156,23299,FOOD COVER WITH BEADS SET 2,6,2011-06-01 07:37:00,3.75,15643.0,United Kingdom
1,555156,22847,BREAD BIN DINER STYLE IVORY,1,2011-06-01 07:37:00,16.95,15643.0,United Kingdom
2,555157,23075,PARLOUR CERAMIC WALL HOOK,16,2011-06-01 07:38:00,4.15,15643.0,United Kingdom
3,555157,47590B,PINK HAPPY BIRTHDAY BUNTING,6,2011-06-01 07:38:00,5.45,15643.0,United Kingdom
4,555157,22423,REGENCY CAKESTAND 3 TIER,4,2011-06-01 07:38:00,12.75,15643.0,United Kingdom


In [54]:
tx_6m.describe()

,Quantity,UnitPrice,CustomerID
count,278966.000000,278966.000000,212734.000000
mean,8.814114,4.179454,15561.650700
std,58.218665,96.550255,1580.590271
min,-9600.000000,-11062.060000,12747.000000
25%,1.000000,1.250000,14194.000000
50%,3.000000,2.080000,15532.000000
75%,10.000000,4.130000,16923.000000
max,12540.000000,38970.000000,18287.000000


In [55]:
#calculate revenue and create a new dataframe for it
tx_6m['Revenue'] = tx_6m['UnitPrice'] * tx_6m['Quantity']
tx_user_6m = tx_6m.groupby('CustomerID')['Revenue'].sum().reset_index()
tx_user_6m.columns = ['CustomerID','m6_Revenue']
tx_user_6m.head()

,CustomerID,m6_Revenue
0,12747.0,1666.11
1,12748.0,18679.01
2,12749.0,2323.04
3,12820.0,561.53
4,12822.0,918.98


In [56]:
tx_user_6m.m6_Revenue.describe()

count      3167.000000
mean       1239.685078
std        4782.390775
min       -4287.630000
25%         257.780000
50%         521.200000
75%        1148.670000
max      180469.050000
Name: m6_Revenue, dtype: float64

In [57]:
#plot LTV histogram
plot_data = [
    go.Histogram(
        x=tx_user_6m.query('m6_Revenue < 10000')['m6_Revenue']
    )
]

plot_layout = go.Layout(
        title='6m Revenue'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
plotly.offline.iplot(fig)

Гистограмма явно показывает, что у нас есть клиенты с отрицательной LTV (пожизненной ценностью клиента). У нас также есть выбросы. Отфильтровать выбросы имеет смысл, чтобы иметь адекватную модель машинного обучения.
Хорошо, следующий шаг. Мы объединим наши наборы данных за 3 месяца и 6 месяцев, чтобы увидеть корреляции между LTV и набором признаков, которыми мы располагаем.

In [58]:
tx_merge = pd.merge(tx_user, tx_user_6m, on='CustomerID', how='left')
tx_merge = tx_merge.fillna(0)
tx_merge.head()

,CustomerID,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster,OverallScore,Segment,m6_Revenue
0,14620.0,12,3,30,0,393.28,0,3,Mid-Value,0.00
1,15194.0,6,3,64,0,1439.02,0,3,Mid-Value,3232.20
2,18044.0,5,3,57,0,808.96,0,3,Mid-Value,991.54
3,18075.0,12,3,35,0,638.12,0,3,Mid-Value,1322.75
4,15241.0,0,3,64,0,947.55,0,3,Mid-Value,791.04


In [59]:
tx_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1840 entries, 0 to 1839
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   CustomerID        1840 non-null   float64
 1   Recency           1840 non-null   int64  
 2   RecencyCluster    1840 non-null   int64  
 3   Frequency         1840 non-null   int64  
 4   FrequencyCluster  1840 non-null   int64  
 5   Revenue           1840 non-null   float64
 6   RevenueCluster    1840 non-null   int64  
 7   OverallScore      1840 non-null   int64  
 8   Segment           1840 non-null   object 
 9   m6_Revenue        1840 non-null   float64
dtypes: float64(3), int64(6), object(1)
memory usage: 158.1+ KB


In [60]:
tx_graph = tx_merge.query("m6_Revenue < 30000")
tx_graph.head()

,CustomerID,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster,OverallScore,Segment,m6_Revenue
0,14620.0,12,3,30,0,393.28,0,3,Mid-Value,0.00
1,15194.0,6,3,64,0,1439.02,0,3,Mid-Value,3232.20
2,18044.0,5,3,57,0,808.96,0,3,Mid-Value,991.54
3,18075.0,12,3,35,0,638.12,0,3,Mid-Value,1322.75
4,15241.0,0,3,64,0,947.55,0,3,Mid-Value,791.04


Код ниже объединяет наши фичи и LTV, строит график LTV vs RFM Score

In [61]:
plot_data = [
    go.Scatter(
        x=tx_graph.query("Segment == 'Low-Value'")['OverallScore'],
        y=tx_graph.query("Segment == 'Low-Value'")['m6_Revenue'],
        mode='markers',
        name='Low',
        marker= dict(size= 7,
            line= dict(width=1),
            color= 'blue',
            opacity= 0.8
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'Mid-Value'")['OverallScore'],
        y=tx_graph.query("Segment == 'Mid-Value'")['m6_Revenue'],
        mode='markers',
        name='Mid',
        marker= dict(size= 9,
            line= dict(width=1),
            color= 'green',
            opacity= 0.5
           )
    ),
        go.Scatter(
        x=tx_graph.query("Segment == 'High-Value'")['OverallScore'],
        y=tx_graph.query("Segment == 'High-Value'")['m6_Revenue'],
        mode='markers',
        name='High',
        marker= dict(size= 11,
            line= dict(width=1),
            color= 'red',
            opacity= 0.9
           )
    ),
]

plot_layout = go.Layout(
        yaxis= {'title': "6m LTV"},
        xaxis= {'title': "RFM Score"},
        title='LTV'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)
plotly.offline.iplot(fig)

Положительная корреляция здесь достаточно заметна. Высокий показатель RFM означает высокую пожизненную ценность клиента (LTV).
Прежде чем строить модель машинного обучения, нам необходимо определить тип этой задачи машинного обучения. Сама LTV представляет собой задачу регрессии. Модель машинного обучения может предсказать денежное значение LTV. Но здесь нам нужны сегменты LTV. Это делает его более действенным и легким для общения с другими людьми. Применяя кластеризацию K-средних, мы можем выявить наши существующие группы LTV и создать на их основе сегменты.
С учетом бизнес-части этого анализа, нам нужно обращаться к клиентам по-разному в зависимости от их предсказанной LTV. В этом примере мы применим кластеризацию и получим 3 сегмента (количество сегментов действительно зависит от динамики и целей вашего бизнеса):
- Низкая LTV (низкая пожизненная ценность клиента)
- Средняя LTV (средняя пожизненная ценность клиента)
- Высокая LTV (высокая пожизненная ценность клиента)

Мы собираемся применить кластеризацию методом K-средних, чтобы определить сегменты и наблюдать за их характеристиками:

In [62]:
#remove outliers
tx_merge = tx_merge[tx_merge['m6_Revenue']<tx_merge['m6_Revenue'].quantile(0.99)]

#creating 3 clusters
kmeans = KMeans(n_clusters=3)
kmeans.fit(tx_merge[['m6_Revenue']])
tx_merge['LTVCluster'] = kmeans.predict(tx_merge[['m6_Revenue']])

#order cluster number based on LTV
tx_merge = order_cluster('LTVCluster', 'm6_Revenue',tx_merge,True)

#creatinga new cluster dataframe
tx_cluster = tx_merge.copy()

#see details of the clusters
tx_cluster.groupby('LTVCluster')['m6_Revenue'].describe()

,count,mean,std,min,25%,50%,75%,max
LTVCluster,,,,,,,,
0,1394.0,396.137189,419.891843,-609.40,0.000,294.220,682.4300,1429.87
1,371.0,2492.794933,937.341566,1445.31,1731.980,2162.930,3041.9550,5287.39
2,56.0,8222.565893,2983.572030,5396.44,6151.435,6986.545,9607.3225,16756.31


2 - это лучший вариант с средней LTV в 8,2 тыс. долларов, в то время как 0 - наихудший с показателем 396.

Перед обучением модели машинного обучения предстоит выполнить несколько дополнительных шагов:
- Необходимо выполнить инжиниринг признаков. Мы должны преобразовать категориальные столбцы в числовые.
- Мы проверим корреляцию признаков с нашей меткой — кластерами LTV.
- Мы разделим наш набор признаков и метку (LTV) на X и y. Мы будем использовать X для предсказания y.

- Создадим набор данных для обучения и тестирования. Набор данных для обучения будет использоваться для построения модели машинного обучения. Мы применим нашу модель к набору данных для тестирования, чтобы оценить ее реальную производительность.

In [63]:
#convert categorical columns to numerical
tx_class = pd.get_dummies(tx_cluster)

#calculate and show correlations
corr_matrix = tx_class.corr()
corr_matrix['LTVCluster'].sort_values(ascending=False)

LTVCluster            1.000000
m6_Revenue            0.845933
Revenue               0.600491
RevenueCluster        0.467191
OverallScore          0.373114
FrequencyCluster      0.366366
Frequency             0.359601
Segment_High-Value    0.352387
RecencyCluster        0.236899
Segment_Mid-Value     0.168473
CustomerID           -0.028401
Recency              -0.237249
Segment_Low-Value    -0.266008
Name: LTVCluster, dtype: float64

In [64]:
#create X and y, X will be feature set and y is the label - LTV
X = tx_class.drop(['LTVCluster','m6_Revenue'],axis=1)
y = tx_class['LTVCluster']

#split training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=56)

In [65]:
X_train.head()

,CustomerID,Recency,RecencyCluster,Frequency,FrequencyCluster,Revenue,RevenueCluster,OverallScore,Segment_High-Value,Segment_Low-Value,Segment_Mid-Value
18,15220.0,11,3,58,0,838.15,0,3,0,0,1
491,15376.0,30,2,59,0,654.17,0,2,0,1,0
1447,16161.0,5,3,70,0,946.46,0,3,0,0,1
323,17345.0,4,3,22,0,128.33,0,3,0,0,1
938,13368.0,64,0,28,0,588.00,0,0,0,1,0


Мы видим, что данные за 3 месяца: выручка, частота покупок и RFM Score будут полезны для наших моделей машинного обучения.

Поскольку у нас есть наборы данных для обучения и тестирования, мы можем построить нашу модель.

In [66]:
#XGBoost Multiclassification Model
ltv_xgb_model = xgb.XGBClassifier(max_depth=5,
                                  num_iterations=100,
                                  learning_rate=0.001,
                                  min_split_loss=0.4,
                                  n_jobs=-1
                                  ).fit(X_train, y_train)

print('Accuracy of XGB classifier on training set: {:.2f}'
       .format(ltv_xgb_model.score(X_train, y_train)))
print('Accuracy of XGB classifier on test set: {:.2f}'
       .format(ltv_xgb_model.score(X_test[X_train.columns], y_test)))

y_pred = ltv_xgb_model.predict(X_test)
print(classification_report(y_test, y_pred))

Accuracy of XGB classifier on training set: 0.83
Accuracy of XGB classifier on test set: 0.89
              precision    recall  f1-score   support

           0       0.91      0.96      0.93        70
           1       0.80      0.67      0.73        18
           2       1.00      0.75      0.86         4

    accuracy                           0.89        92
   macro avg       0.90      0.79      0.84        92
weighted avg       0.89      0.89      0.89        92



Точность и полнота приемлемы для кластера 0. В качестве примера, для кластера 0 (Низкая LTV), если модель говорит нам, что этот клиент принадлежит к кластеру 0, то 91 из 100 будут правильными (precision). И модель успешно идентифицирует 96% реальных клиентов из кластера 0 (recall). Нам действительно нужно улучшить модель для других кластеров. Например, мы едва ли обнаруживаем 67% клиентов с средним LTV. Возможные действия для улучшения ситуации:
- Добавление больше признаков и улучшение инжиниринга признаков.
- Попробовать другие модели, кроме XGBoost.
- Применить настройку гиперпараметров к текущей модели.
- Добавить больше данных в модель, если это возможно.


Отлично! Теперь у нас есть модель машинного обучения, которая предсказывает будущие сегменты LTV наших клиентов. Мы легко можем адаптировать наши действия на основе этого. Например, нам определенно не хочется потерять клиентов с высокой LTV.

reference:
https://towardsdatascience.com/data-driven-growth-with-python-part-3-customer-lifetime-value-prediction-6017802f2e0f
https://www.kaggle.com/code/shailaja4247/customer-lifetime-value-prediction
